# BIG DATA ANALYSIS : 이상탐지 실습 1
---


## 1. 관련된 라이브러리 Import 하기

In [ ]:
# Author: Alexandre Gramfort <alexandre.gramfort@inria.fr>
#         Albert Thomas <albert.thomas@telecom-paristech.fr>
# License: BSD 3 clause

# numpy 및 matplot lib
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


# 이상탐지 알고리즘들 추가
from sklearn import svm
from sklearn.datasets import make_moons, make_blobs
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

## 2. 토이데이터 생성

In [ ]:
# 전체 데이터 수
n_samples = 300
# 이상치 비율
outliers_fraction = 0.1
# 이상 데이터 수
n_outliers = int(outliers_fraction * n_samples)
# 정상 데이터 수
n_inliers = n_samples - n_outliers

rng = np.random.RandomState(42)
# 특별한 패턴의 정상 데이터셋 제작
blobs_params = dict(random_state=0, n_samples=n_inliers, n_features=2)
datasets = [
    make_blobs(centers=[[0, 0], [0, 0]], cluster_std=0.5,
               **blobs_params)[0],
    make_blobs(centers=[[2, 2], [-2, -2]], cluster_std=[0.5, 0.5],
               **blobs_params)[0],
    make_blobs(centers=[[2, 2], [-2, -2]], cluster_std=[1.5, .3],
               **blobs_params)[0],
    4. * (make_moons(n_samples=n_inliers, noise=.05, random_state=0)[0] -
          np.array([0.5, 0.25])),
    14. * (np.random.RandomState(42).rand(n_inliers, 2) - 0.5)]


for dataset in datasets:
    print(dataset.shape)


## 3. 토이데이터 확인

In [ ]:
plot_num = 1
plt.figure(figsize=(20, 4))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)
for X in datasets:
    
    plt.subplot(1,len(datasets), plot_num)
    plt.scatter(X[:, 0], X[:, 1], s=10)
    plt.xlim(-7, 7)
    plt.ylim(-7, 7)
    plot_num+=1

## 4. 이상치 데이터 생성

In [ ]:
outliers = rng.uniform(low=-6, high=6, size=(n_outliers, 2))
plt.scatter(outliers[:, 0], outliers[:, 1], s=10, c="red")


## 5. 이상탐지 알고리즘 정의

In [ ]:
anomaly_algorithms = [
    ("Robust covariance", EllipticEnvelope(contamination=outliers_fraction)),
    ("One-Class SVM", svm.OneClassSVM(nu=outliers_fraction, kernel="rbf",
                                      gamma=0.1)),
    ("Isolation Forest", IsolationForest(contamination=outliers_fraction,
                                         random_state=42)),
    ("Local Outlier Factor", LocalOutlierFactor(
        n_neighbors=35, contamination=outliers_fraction))]

In [ ]:
# 예측 영역을 그리기 위한 점들
xx, yy = np.meshgrid(np.linspace(-7, 7, 150),
                     np.linspace(-7, 7, 150))

## 6. 이상탐지 결과 비교

In [ ]:


# Plot 모양 세팅
plt.figure(figsize=(len(anomaly_algorithms) * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1
#초기화값 고정


for i_dataset, X in enumerate(datasets):
    # 아웃라이어들을 정상 데이터에 추가
    X = np.concatenate([X, rng.uniform(low=-6, high=6,
                       size=(n_outliers, 2))], axis=0)

    for name, algorithm in anomaly_algorithms:
        t0 = time.time()
        algorithm.fit(X)
        t1 = time.time()
        plt.subplot(len(datasets), len(anomaly_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        # 알고리즘별 결과를 출력하는 함수가 다름
        if name == "Local Outlier Factor":
            y_pred = algorithm.fit_predict(X)
        else:
            y_pred = algorithm.fit(X).predict(X)

        # 정상영역 출력
        if name != "Local Outlier Factor":  # LOF는 예측 기능이 없기 때문에 생략
            Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)

            plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='black')

        colors = np.array(['#377eb8', '#ff7f00'])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[(y_pred + 1) // 2])

        plt.xlim(-7, 7)
        plt.ylim(-7, 7)
        plt.xticks(())
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        plot_num += 1

plt.show()